<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [2]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Accuracy
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [3]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 72.17%
[2,000] Accuracy: 71.74%
[3,000] Accuracy: 72.22%
[4,000] Accuracy: 73.34%
[5,000] Accuracy: 73.59%
[6,000] Accuracy: 73.88%
[7,000] Accuracy: 74.44%
[8,000] Accuracy: 74.26%
[9,000] Accuracy: 74.25%
[10,000] Accuracy: 74.35%
[11,000] Accuracy: 74.75%
[12,000] Accuracy: 74.94%
[13,000] Accuracy: 75.02%
[14,000] Accuracy: 74.92%
[15,000] Accuracy: 75.13%
[16,000] Accuracy: 75.09%
[17,000] Accuracy: 75.13%
[18,000] Accuracy: 74.90%


Accuracy: 74.91%

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [4]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 75.78%
[2,000] Accuracy: 77.14%
[3,000] Accuracy: 78.16%
[4,000] Accuracy: 77.89%
[5,000] Accuracy: 77.70%
[6,000] Accuracy: 77.61%
[7,000] Accuracy: 77.88%
[8,000] Accuracy: 77.62%
[9,000] Accuracy: 77.50%
[10,000] Accuracy: 77.58%
[11,000] Accuracy: 77.80%
[12,000] Accuracy: 77.93%
[13,000] Accuracy: 78.05%
[14,000] Accuracy: 77.78%
[15,000] Accuracy: 77.89%
[16,000] Accuracy: 77.89%
[17,000] Accuracy: 77.91%
[18,000] Accuracy: 77.71%


Accuracy: 77.73%

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [5]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 73.57%
[2,000] Accuracy: 76.04%
[3,000] Accuracy: 77.29%
[4,000] Accuracy: 77.12%
[5,000] Accuracy: 77.24%
[6,000] Accuracy: 77.35%
[7,000] Accuracy: 77.73%
[8,000] Accuracy: 77.60%
[9,000] Accuracy: 77.43%
[10,000] Accuracy: 77.46%
[11,000] Accuracy: 77.70%
[12,000] Accuracy: 78.01%
[13,000] Accuracy: 78.01%
[14,000] Accuracy: 77.79%
[15,000] Accuracy: 77.97%
[16,000] Accuracy: 78.01%
[17,000] Accuracy: 77.98%
[18,000] Accuracy: 77.71%


Accuracy: 77.70%

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [7]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 74.37%
[2,000] Accuracy: 76.74%
[3,000] Accuracy: 77.89%
[4,000] Accuracy: 78.17%
[5,000] Accuracy: 78.16%
[6,000] Accuracy: 78.25%
[7,000] Accuracy: 78.73%
[8,000] Accuracy: 78.22%
[9,000] Accuracy: 78.22%
[10,000] Accuracy: 78.35%
[11,000] Accuracy: 78.59%
[12,000] Accuracy: 78.81%
[13,000] Accuracy: 78.74%
[14,000] Accuracy: 78.46%
[15,000] Accuracy: 78.59%
[16,000] Accuracy: 78.58%
[17,000] Accuracy: 78.61%
[18,000] Accuracy: 78.49%


Accuracy: 78.52%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [3]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [11]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 79.58%
[2,000] Accuracy: 85.59%
[3,000] Accuracy: 86.50%
[4,000] Accuracy: 88.30%
[5,000] Accuracy: 88.66%
[6,000] Accuracy: 84.06%
[7,000] Accuracy: 82.23%
[8,000] Accuracy: 81.16%
[9,000] Accuracy: 80.15%
[10,000] Accuracy: 78.89%
[11,000] Accuracy: 77.28%
[12,000] Accuracy: 77.01%
[13,000] Accuracy: 77.15%
[14,000] Accuracy: 77.48%
[15,000] Accuracy: 77.77%
[16,000] Accuracy: 76.94%
[17,000] Accuracy: 76.58%
[18,000] Accuracy: 76.46%
[19,000] Accuracy: 76.27%
[20,000] Accuracy: 75.89%


Accuracy: 75.89%

## Leveraging Bagging

In [12]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 75.48%
[2,000] Accuracy: 79.34%
[3,000] Accuracy: 81.93%
[4,000] Accuracy: 83.95%
[5,000] Accuracy: 85.30%
[6,000] Accuracy: 80.80%
[7,000] Accuracy: 78.00%
[8,000] Accuracy: 76.46%
[9,000] Accuracy: 75.32%
[10,000] Accuracy: 74.37%
[11,000] Accuracy: 72.86%
[12,000] Accuracy: 73.75%
[13,000] Accuracy: 74.65%
[14,000] Accuracy: 75.74%
[15,000] Accuracy: 76.64%
[16,000] Accuracy: 75.63%
[17,000] Accuracy: 75.29%
[18,000] Accuracy: 75.10%
[19,000] Accuracy: 74.96%
[20,000] Accuracy: 74.71%


Accuracy: 74.71%

## AdaptiveRandomForest

In [4]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 65.27%
[2,000] Accuracy: 65.88%
[3,000] Accuracy: 66.96%
[4,000] Accuracy: 68.72%
[5,000] Accuracy: 69.35%
[6,000] Accuracy: 65.91%
[7,000] Accuracy: 64.99%
[8,000] Accuracy: 64.51%
[9,000] Accuracy: 64.50%
[10,000] Accuracy: 64.37%
[11,000] Accuracy: 63.12%
[12,000] Accuracy: 64.09%
[13,000] Accuracy: 65.18%
[14,000] Accuracy: 66.16%
[15,000] Accuracy: 66.91%
[16,000] Accuracy: 66.22%
[17,000] Accuracy: 66.22%
[18,000] Accuracy: 66.46%
[19,000] Accuracy: 66.78%
[20,000] Accuracy: 66.97%


Accuracy: 66.97%

## StreamingRandomPatches
---
We set the drift and warning detection options

In [15]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 92.09%
[2,000] Accuracy: 93.25%
[3,000] Accuracy: 93.43%
[4,000] Accuracy: 93.80%
[5,000] Accuracy: 93.84%
[6,000] Accuracy: 88.25%
[7,000] Accuracy: 84.68%
[8,000] Accuracy: 81.96%
[9,000] Accuracy: 80.01%
[10,000] Accuracy: 78.35%
[11,000] Accuracy: 77.22%
[12,000] Accuracy: 77.56%
[13,000] Accuracy: 78.04%
[14,000] Accuracy: 78.84%
[15,000] Accuracy: 79.33%
[16,000] Accuracy: 78.67%
[17,000] Accuracy: 78.55%
[18,000] Accuracy: 78.56%
[19,000] Accuracy: 78.64%
[20,000] Accuracy: 78.51%


Accuracy: 78.51%

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.